In [1]:
import numpy as np
import tensorflow as tf
import random
from sklearn.metrics import confusion_matrix
from tensorflow.contrib import rnn

num_epoch = 100
learn_rate = 0.001 #0.1,0.05
batch_size = 300 #1000,2000,3000
keep_rate = 0.9 #0.8,0.7,0.6,0.5
num_label = 10
num_feature =  315 #96
# padding_size=8
num_step_window = 20
epsilon=1e-3
label_dict = {}
cell_size=128  #128
n_step=1
num_layer=2

tf.reset_default_graph()
import sys
sys.path.append('./')

class BatchFetcher:
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.dataPointer = 0
        self.dataSize = len(self.labels)

        self.label_dict = {}
        label_list = []
        for i in range(0, 10):
            label_list.append(i)
        for i in range(num_label):
            l = label_list[i]
            self.label_dict[l] = i

    def onehotEncoder(self, labels):
        new_labels = np.zeros(shape=(len(labels), num_label), dtype=np.int)
        for i in range(len(labels)):
            l = labels[i]
            index = self.label_dict[l]
            new_labels[i][index] = 1
        return new_labels

    def next_batch(self):
        left = self.dataPointer
        end_epoch = False
        if self.dataPointer == 0:
            labels = np.reshape(self.labels, newshape=(self.dataSize, 1))
            X = np.concatenate((self.features, labels), axis=1)
            #random.shuffle(X)
            self.labels = np.array(X[:, -1], dtype=np.int)
            self.features = X[:, :-1]

        # fetcher right bound
        if self.dataPointer + 2 * batch_size > self.dataSize:
            right = self.dataSize
        else:
            right = left + batch_size

        feature_batch = self.features[left:right]
        label_batch = self.labels[left:right]
        label_batch = self.onehotEncoder(label_batch)
        # move pointer
        self.dataPointer = right
        if self.dataPointer == self.dataSize:
            self.dataPointer = 0
            end_epoch = True
        return feature_batch, label_batch, end_epoch


def loadData():
    traindata = []
    testdata = []
    
    trainfile = './NB15-TAM-Train-minmax.csv'
    testfile = './NB15-TAM-Test-minmax.csv'

    with open(trainfile, "r") as f:
        for line in f:
            x = np.fromstring(line, dtype=np.float, sep=',')
            traindata.append(x)
    traindata = np.array(traindata)

    with open(testfile, "r") as f:
        for line in f:
            x = np.fromstring(line, dtype=np.float, sep=',')
            testdata.append(x)
    testdata = np.array(testdata)

    trainLabels = np.array(traindata[:, -1], dtype=np.int)
    trainfeatures = np.array(traindata[:, 0:315])
    testLabels = np.array(testdata[:, -1], dtype=np.int)
    testfeatures = np.array(testdata[:, 0:315])

    print("\n=======Loaded Data=========\n")
    print("train/test(labels,features)",
          trainLabels.shape, trainfeatures.shape, testLabels.shape, testfeatures.shape)
    import collections
    labels_train, labels_test = collections.Counter(trainLabels), collections.Counter(testLabels)
    print("label num (train/test)", len(labels_train), len(labels_test))
    print(labels_train)
    print(labels_test)
    print("\n=================\n")
    return trainfeatures, trainLabels, testfeatures, testLabels

def weight_init_1(size):
    r = tf.contrib.layers.l2_regularizer(1e-4)
    return tf.get_variable('w1_xaiver',size, initializer=tf.contrib.layers.xavier_initializer(),regularizer=r)
def weight_init_2(size):
    r = tf.contrib.layers.l2_regularizer(1e-4)
    return tf.get_variable('w2_xaiver',size, initializer=tf.contrib.layers.xavier_initializer(),regularizer=r)
def weight_3(size):
    r = tf.contrib.layers.l2_regularizer(1e-4)
    return tf.get_variable('w3_xaiver',size, initializer=tf.contrib.layers.xavier_initializer(),regularizer=r)
def weight_4(size):
    r = tf.contrib.layers.l2_regularizer(1e-4)
    return tf.get_variable('w4_xaiver',size, initializer=tf.contrib.layers.xavier_initializer(),regularizer=r)
def weight_5(size):
    r = tf.contrib.layers.l2_regularizer(1e-4)
    return tf.get_variable('w5_xaiver',size, initializer=tf.contrib.layers.xavier_initializer(),regularizer=r)

def weight1_1(size):
    return tf.get_variable('x1_xaiver',size,initializer=tf.contrib.layers.xavier_initializer())
def weight1_2(size):
    return tf.get_variable('x2_xaiver',size,initializer=tf.contrib.layers.xavier_initializer())
def weight1_3(size):
    return tf.get_variable('x3_xaiver',size,initializer=tf.contrib.layers.xavier_initializer())
def weight1_4(size):
    return tf.get_variable('x4_xaiver',size,initializer=tf.contrib.layers.xavier_initializer())
def bias(size):
    return tf.Variable(tf.constant(0.1,shape=size))
def conv(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
def confusion(actual, predict):
    y_actual = np.argmax(actual, axis=1)
    y_predict = np.argmax(predict, axis=1)
    array = confusion_matrix(y_actual, y_predict)
    TP = np.sum(array[0, 0])
    TN = np.sum(array[1:, 1:])
    FN = np.sum(array[0, 1:])
    FP = np.sum(array[1:, 0])
    tp_true = array[1, 1] + array[2, 2]+ array[3, 3]+ array[4, 4]+ array[5, 5]+array[6, 6]+array[7, 7]+array[8, 8]+array[9, 9]
    return TP, TN, FN, FP,tp_true,array
def LSTM(x,keep_prob):
    global cell_init_state
    global cell_final_state

    l_in_x=tf.reshape(x,[-1,num_feature])
    l_in_w=weight_init_1([num_feature,cell_size])
    # l_in_x=tf.reshape(x,[-1,num_feature])
    # l_in_w=weight_init_1([num_feature,cell_size])
    l_in_b=bias([cell_size])
    l_in_y=tf.reshape(tf.matmul(l_in_x,l_in_w)+l_in_b,[-1,n_step,cell_size])

    # cell=rnn.BasicRNNCell(cell_size)
    cell = rnn.BasicLSTMCell(cell_size, forget_bias=1.0, state_is_tuple=True)
    cell =  rnn.DropoutWrapper(cell,output_keep_prob=keep_prob)
    cells = rnn.MultiRNNCell([cell for _ in range(num_layer)])
    # cells = rnn.MultiRNNCell(cell_size)
    cell_output, cell_final_state = tf.nn.dynamic_rnn(cells, l_in_y, dtype=tf.float32, time_major=False)
    t=tf.reshape(cell_output,[-1,cell_size])
    l_out_x=tf.reshape(cell_output,[-1,cell_size])
    l_out_w = weight_init_2([cell_size,96])
    l_out_b = bias([96])
    # result = tf.nn.softmax(tf.matmul(l_out_x, l_out_w) + l_out_b)
    result = tf.matmul(l_out_x, l_out_w) + l_out_b
    return result

def dnn(xs,keep_prob):
    w1=weight_3([96+96+315,128])
    b1=bias([128])
    l1=tf.nn.relu(tf.matmul(xs,w1)+b1)
    l_1_d = tf.nn.dropout(l1, keep_prob)

    w2 = weight_4([128, 256])
    b2 = bias([256])
    l2 = tf.nn.relu(tf.matmul(l_1_d, w2) + b2)
    l_2_d = tf.nn.dropout(l2, keep_prob)

    w3 = weight_5([256, 10])
    b3 = bias([10])
    l3=tf.matmul(l_2_d, w3) + b3
    #l3 = tf.nn.softmax(tf.matmul(l_1_d, w3) + b3)

    return l3


def runModel():
    xs = tf.placeholder(tf.float32, [None, num_feature])
    ys = tf.placeholder(tf.float32, [None, num_label])
    keep_prob = tf.placeholder(tf.float32)
    x_in = tf.reshape(xs, [-1, 15, 21, 1])

    w1 = weight1_1([3, 3, 1, 64]) #64,128,256
    b1 = bias([64])
    cov1 = conv(x_in, w1) + b1
    scale1 = tf.Variable(tf.ones([64]))
    beta1 = tf.Variable(tf.zeros([64]))
    batch_mean1, batch_var1 = tf.nn.moments(cov1, [0, 1, 2])
    bn1 = tf.nn.batch_normalization(cov1, batch_mean1, batch_var1, beta1, scale1, epsilon)
    l1 = max_pool(tf.nn.relu(bn1))

    w2 = weight1_2([3, 3, 64, 64]) #64,128,256
    b2 = bias([64])
    cov2 = conv(l1, w2) + b2
    scale2 = tf.Variable(tf.ones([64]))
    beta2 = tf.Variable(tf.zeros([64]))
    batch_mean2, batch_var2 = tf.nn.moments(cov2, [0, 1, 2])
    bn2 = tf.nn.batch_normalization(cov2, batch_mean2, batch_var2, beta2, scale2, epsilon)
    l2 = max_pool(tf.nn.relu(bn2))

    w3 = weight1_3([3, 3, 64, 16]) #64,128,256
    b3 = bias([16])
    cov3 = conv(l2, w3) + b3
    scale3 = tf.Variable(tf.ones([16]))
    beta3 = tf.Variable(tf.zeros([16]))
    batch_mean3, batch_var3 = tf.nn.moments(cov3, [0, 1, 2])
    bn3 = tf.nn.batch_normalization(cov3, batch_mean3, batch_var3, beta3, scale3, epsilon)
    l3 = max_pool(tf.nn.relu(bn3))
    flat = tf.reshape(l3, [-1, 16*6])
    #fea = LSTM(xs,keep_prob)
    
    #----------------#CNN-LSTM
    #fea = LSTM(flat,keep_prob)
    #prediction=dnn(fea,keep_prob)
    
    #----------------#CNN-LSTM-TAM
    fea = LSTM(xs,keep_prob)
    flat = tf.concat([flat, fea], 1)
    flat = tf.concat([flat, xs], 1)
    
    #----------------#CNN-LSTM-Attention
    #scale4_1 = tf.Variable(tf.ones([96]))
    #beta4_1 = tf.Variable(tf.zeros([96]))
    #batch_mean4_1, batch_var4_1 = tf.nn.moments(flat, [0, 1])
    #y_l = tf.nn.batch_normalization(flat, batch_mean4_1, batch_var4_1, beta4_1, scale4_1, epsilon)

    #scale4_2 = tf.Variable(tf.ones([96]))
    #beta4_2 = tf.Variable(tf.zeros([96]))
    #batch_mean4_2, batch_var4_2 = tf.nn.moments(fea, [0, 1])
    #y_s = tf.nn.batch_normalization(fea, batch_mean4_2, batch_var4_2, beta4_2, scale4_2, epsilon)
    #flat = y_l * flat + y_s * fea

    prediction=dnn(flat,keep_prob)

    loss = tf.losses.softmax_cross_entropy(ys, prediction)
    train_step = tf.train.AdamOptimizer(learn_rate).minimize(loss)
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(num_epoch):
            end_epoch = False
            step = 0
            while end_epoch == False:
                step += 1
                feature_batch, label_batch, end_epoch = batcher.next_batch()
                sess.run(train_step, feed_dict={xs: feature_batch, ys: label_batch, keep_prob: keep_rate})

                # if step % 20 == 0:
                #     print("#%d,train loss" %step, sess.run(loss, feed_dict={xs: feature_batch, ys: label_batch, keep_prob: 1}))

            print("\n==========finished epoch:%d==========进行测试集预测" % (epoch + 1))

            predicts = sess.run(prediction,feed_dict={xs: testfeatures, ys: testLabels, keep_prob: 1})
            ylabel = sess.run(ys, feed_dict={ys: testLabels})
            TP, TN, FN, FP,tp_true,array = confusion(ylabel, predicts)
            print(array)
            print("test label(%d) accuracy" % num_label, sess.run(accuracy,feed_dict={xs: testfeatures, ys: testLabels,keep_prob: 1}))


if __name__ == '__main__':
    trainfeatures, trainLabels, testfeatures, testLabels = loadData()

    batcher = BatchFetcher(trainfeatures, trainLabels)

    testLabels = batcher.onehotEncoder(testLabels)

    runModel()




=======Loaded Data=========

train/test(labels,features) (175341,) (175341, 315) (82332,) (82332, 315)
label num (train/test) 10 10
Counter({0: 56000, 2: 40000, 1: 33393, 3: 18184, 5: 12264, 4: 10491, 6: 2000, 7: 1746, 8: 1133, 9: 130})
Counter({0: 37000, 2: 18871, 1: 11132, 3: 6062, 5: 4089, 4: 3496, 6: 677, 7: 583, 8: 378, 9: 44})




W0714 14:46:36.279697 139910509823744 deprecation.py:323] From <ipython-input-1-7fd97f53f000>:162: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0714 14:46:36.283671 139910509823744 deprecation.py:323] From <ipython-input-1-7fd97f53f000>:164: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0714 14:46:36.285076 139910509823744 rnn_cell_impl.py:1642] At least two cells provided to MultiRNNCell are the same object and will share weights.
W0714 14:46:36.286399 139910509823744 deprecation.py:323] From <ipython-input-1-7fd97f53f000>:166: dynamic_rnn (from tensorflow.python.ops.rnn) is 


==========finished epoch:1==========进行测试集预测


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[82332,64,15,21] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node moments/SquaredDifference (defined at <ipython-input-1-7fd97f53f000>:205) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[82332,64,15,21] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node moments/SquaredDifference (defined at <ipython-input-1-7fd97f53f000>:205) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[add_7/_71]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions f